In [ ]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
#     print(collections.Counter(ArousalFile))
#     print(collections.Counter(ValenceFile))
#     print(np.mean(ValenceFile))
#     print(np.mean(ArousalFile))
    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
#     print("Accuracy score of Valence ")
#     print(accuracy_score(y_test, y_predict)*100)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
#     print(z_test)
#     print(z_predict)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]

    
# print("$$$$$$$Arousal$$$$$$$")

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]



def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]

def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    for i in range (100):
        svmpca=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svmpca[0]]
        valence+=[svmpca[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Combined "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined)
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Happy and Sad "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined);

    
if __name__ == "__main__":
    HS=0;
    Combined=0;
    Valence=0
    Arousal=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    print("Average happy sad for first 9 "+str(HS/9))
    print("Average Combined sad for first 9 "+str(Combined/9))
    print("Average Valence for first 9 "+str(Valence/9))
    print("Average Arousal for first 9 "+str(Arousal/9))
    for i in range(14,33):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]

s01
Counter({40.0: 43, 60.0: 41, 20.0: 11, 80.0: 5})
Happy Sad
48.0
Counter({25.0: 32, 37.5: 28, 12.5: 15, 50.0: 12, 62.5: 7, 0.0: 4, 75.0: 2})
Combined Emotions
32.25
Arousal 58.75
Counter({62.5: 32, 50.0: 23, 75.0: 23, 37.5: 11, 25.0: 6, 87.5: 5})
Valence 36.625
Counter({37.5: 48, 25.0: 28, 50.0: 16, 62.5: 3, 0.0: 2, 12.5: 1, 87.5: 1, 75.0: 1})
Arousal Valence
32.25
s02
Counter({80.0: 48, 100.0: 30, 60.0: 21, 40.0: 1})
Happy Sad
81.4
Counter({50.0: 32, 62.5: 25, 37.5: 23, 25.0: 10, 75.0: 7, 0.0: 2, 12.5: 1})
Combined Emotions
48.125
Arousal 65.75
Counter({62.5: 33, 75.0: 33, 50.0: 13, 87.5: 10, 37.5: 7, 25.0: 2, 100.0: 2})
Valence 74.0
Counter({75.0: 31, 87.5: 27, 62.5: 21, 50.0: 11, 100.0: 8, 25.0: 2})
Arousal Valence
48.125
s03
Counter({33.33333333333333: 58, 0.0: 21, 66.66666666666666: 18, 100.0: 3})
Happy Sad
34.33333333333333
Counter({62.5: 28, 75.0: 26, 37.5: 15, 50.0: 15, 87.5: 11, 25.0: 3, 0.0: 1, 100.0: 1})
Combined Emotions
61.5
Arousal 76.75
Counter({75.0: 35, 87.5: 29, 62